In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Fasttext and torch
import fasttext
import fasttext.util
import torch

# Data Loader and Processor
import loader
from DataProcessor import DataInstance, DataProcesser

# Import numpy
import numpy as np

# Utils
import tools
import pickle

participants = loader.loadDataParticipants()

ftDims = 300

# Loads fasttext, have a fasttext embeddins in directory, like cc.da.300.bin
ft = loader.LoadWordEmbeddings(dim = ftDims) # Can take a long time to load

In [ ]:
def encodeDocs(docs):
    nDocs = []
    for doc in docs:
        doc = tools.getMoreSent(doc)
        doc = tools.rawTokenize(doc)
        doc = tools.EncodeFT(ft, doc)
        nDocs.append(doc)
    return nDocs

def createLabels(num = 18):
    labels = []
    for i in range(num):
        l = np.zeros(num)
        l[i] = 1
        labels.append(l)
    return labels
    
def zipDataset(labels, data):
    dataset = []
    for l, d in zip(labels, data):
        for p in d:
            inst = DataInstance([], [], [], label = torch.tensor(l))
            inst.preds = torch.tensor(p)
            dataset.append(inst)
        
    return dataset

In [ ]:
data = encodeDocs(participants)
labels = createLabels(len(participants))
dataset = zipDataset(labels, data)

In [ ]:
Data = DataProcesser.FromInstances(dataset)

In [ ]:
Data.save(to = './processed-fasttext-raw.bin')